# Tuning Neural Networks with Normalization - Lab 

## Introduction

In this lab you'll build a neural network to perform a regression task.

It is worth noting that getting regression to work with neural networks can be comparatively difficult because the output is unbounded ($\hat y$ can technically range from $-\infty$ to $+\infty$), and the models are especially prone to exploding gradients. This issue makes a regression exercise the perfect learning case for tinkering with normalization and optimization strategies to ensure proper convergence!

## Objectives

In this lab you will: 

- Fit a neural network to normalized data 
- Implement and observe the impact of various initialization techniques 
- Implement and observe the impact of various optimization techniques 

## Load the data 

First, run the following cell to import all the neccessary libraries and classes you will need in this lab. 

In [9]:
# Necessary libraries and classes
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras import initializers
from keras import layers
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from keras import optimizers
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

In this lab, you'll be working with the housing prices data you saw in an earlier section. However, we did a lot of preprocessing for you so you can focus on normalizing numeric features and building neural network models! The following preprocessing steps were taken (all the code can be found in the `data_preprocessing.ipynb` notebook in this repository): 

- The data was split into the training, validate, and test sets 
- All the missing values in numeric columns were replaced by the median of those columns 
- All the missing values in catetgorical columns were replaced with the word 'missing' 
- All the categorical columns were one-hot encoded 

Run the following cells to import the train, validate, and test sets:  

In [10]:
# Load all numeric features
X_train_numeric = pd.read_csv('data/X_train_numeric.csv')
X_val_numeric = pd.read_csv('data/X_val_numeric.csv')
X_test_numeric = pd.read_csv('data/X_test_numeric.csv')

# Load all categorical features
X_train_cat = pd.read_csv('data/X_train_cat.csv')
X_val_cat = pd.read_csv('data/X_val_cat.csv')
X_test_cat = pd.read_csv('data/X_test_cat.csv')

# Load all targets
y_train = pd.read_csv('data/y_train.csv')
y_val = pd.read_csv('data/y_val.csv')
y_test = pd.read_csv('data/y_test.csv')

In [11]:
# Combine all features
X_train = pd.concat([X_train_numeric, X_train_cat], axis=1)
X_val = pd.concat([X_val_numeric, X_val_cat], axis=1)
X_test = pd.concat([X_test_numeric, X_test_cat], axis=1)

# Number of features
n_features = X_train.shape[1]

As a refresher, preview the training data: 

In [12]:
# Preview the data
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,80.0,69.0,21453.0,6.0,5.0,1969.0,1969.0,0.0,938.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,60.0,79.0,12420.0,7.0,5.0,2001.0,2001.0,0.0,666.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,20.0,75.0,9742.0,8.0,5.0,2002.0,2002.0,281.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,120.0,39.0,5389.0,8.0,5.0,1995.0,1996.0,0.0,1180.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,60.0,85.0,11003.0,10.0,5.0,2008.0,2008.0,160.0,765.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


## Build a Baseline Model

Building a naive baseline model to compare performance against is a helpful reference point. From there, you can then observe the impact of various tunning procedures which will iteratively improve your model. So, let's do just that! 

In the cell below: 

- Add an input layer with `n_features` units 
- Add two hidden layers, one with 100 and the other with 50 units (make sure you use the `'relu'` activation function) 
- Add an output layer with 1 unit and `'linear'` activation 
- Compile and fit the model 

In [13]:
np.random.seed(123)
baseline_model = Sequential()

# Hidden layer with 100 units
baseline_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))

# Hidden layer with 50 units
baseline_model.add(layers.Dense(50, activation='relu'))

# Output layer
baseline_model.add(layers.Dense(1, activation='linear'))

# Compile the model
baseline_model.compile(optimizer='SGD', 
                       loss='mse', 
                       metrics=['mse'])

# Train the model
baseline_model.fit(X_train, 
                   y_train, 
                   batch_size=32, 
                   epochs=150, 
                   validation_data=(X_val, y_val))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 1s 653us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 2/150
1051/1051 [==============================] - 1s 507us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 3/150
1051/1051 [==============================] - 0s 265us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 4/150
1051/1051 [==============================] - 0s 319us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 5/150
1051/1051 [==============================] - 0s 318us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 6/150
1051/1051 [==============================] - 0s 229us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 7/150
1051/1051 [

Epoch 53/150
1051/1051 [==============================] - 0s 280us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 54/150
1051/1051 [==============================] - 0s 284us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 55/150
1051/1051 [==============================] - 0s 162us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 56/150
1051/1051 [==============================] - 0s 219us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 57/150
1051/1051 [==============================] - 0s 159us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 58/150
1051/1051 [==============================] - 0s 222us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 59/150
1051/1051 [==============================] - 0s 189

1051/1051 [==============================] - 0s 207us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 106/150
1051/1051 [==============================] - 0s 205us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 107/150
1051/1051 [==============================] - 0s 190us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 108/150
1051/1051 [==============================] - 0s 219us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 109/150
1051/1051 [==============================] - 0s 281us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 110/150
1051/1051 [==============================] - 0s 357us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 111/150
1051/1051 [==============================] - 0s 361us/step

> _**Notice this extremely problematic behavior: all the values for training and validation loss are "nan". This indicates that the algorithm did not converge. The first solution to this is to normalize the input. From there, if convergence is not achieved, normalizing the output may also be required.**_ 

## Normalize the Input Data 

It's now time to normalize the input data. In the cell below: 

- Assign the column names of all numeric columns to `numeric_columns` 
- Instantiate a `StandardScaler` 
- Fit and transform `X_train_numeric`. Make sure you convert the result into a DataFrame (use `numeric_columns` as the column names) 
- Transform validate and test sets (`X_val_numeric` and `X_test_numeric`), and convert these results into DataFrames as well 
- Use the provided to combine the scaled numerical and categorical features 

In [14]:
# Numeric column names
numeric_columns = X_train_numeric.columns 

# Instantiate StandardScaler
ss_X = StandardScaler()

# Fit and transform train data
X_train_scaled = pd.DataFrame(ss_X.fit_transform(X_train_numeric), columns=numeric_columns)

# Transform validate and test data
X_val_scaled = pd.DataFrame(ss_X.transform(X_val_numeric), columns=numeric_columns)
X_test_scaled = pd.DataFrame(ss_X.transform(X_test_numeric), columns=numeric_columns)

# Combine the scaled numerical features and categorical features
X_train = pd.concat([X_train_scaled, X_train_cat], axis=1)
X_val = pd.concat([X_val_scaled, X_val_cat], axis=1)
X_test = pd.concat([X_test_scaled, X_test_cat], axis=1)

Now run the following cell to compile a neural network model (with the same architecture as before): 

In [15]:
# Model with all normalized inputs
np.random.seed(123)
normalized_input_model = Sequential()
normalized_input_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
normalized_input_model.add(layers.Dense(50, activation='relu'))
normalized_input_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_input_model.compile(optimizer='SGD', 
                               loss='mse', 
                               metrics=['mse'])

In the cell below: 
- Train the `normalized_input_model` on normalized input (`X_train`) and output (`y_train`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` argument as `(X_val, y_val)` 

In [16]:
# Train the model
normalized_input_model.fit(X_train,  
                           y_train, 
                           batch_size=32, 
                           epochs=150, 
                           validation_data=(X_val, y_val))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 1s 683us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan        
Epoch 2/150
1051/1051 [==============================] - 0s 445us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 3/150
1051/1051 [==============================] - 0s 362us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 4/150
1051/1051 [==============================] - 0s 292us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 5/150
1051/1051 [==============================] - 0s 454us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 6/150
1051/1051 [==============================] - 0s 254us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 7/150
105

1051/1051 [==============================] - 0s 383us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 54/150
1051/1051 [==============================] - 0s 239us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 55/150
1051/1051 [==============================] - 0s 303us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 56/150
1051/1051 [==============================] - 0s 196us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 57/150
1051/1051 [==============================] - 0s 317us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 58/150
1051/1051 [==============================] - 0s 320us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 59/150
1051/1051 [==============================] - 0s 215us/step - los

1051/1051 [==============================] - 0s 221us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 106/150
1051/1051 [==============================] - 0s 192us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 107/150
1051/1051 [==============================] - 0s 219us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 108/150
1051/1051 [==============================] - 0s 195us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 109/150
1051/1051 [==============================] - 0s 234us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 110/150
1051/1051 [==============================] - 0s 186us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 111/150
1051/1051 [==============================] - 0s 202us/step

> _**Note that you still haven't achieved convergence! From here, it's time to normalize the output data.**_

## Normalizing the output

Again, use `StandardScaler()` to: 

- Fit and transform `y_train` 
- Transform `y_val` and `y_test` 

In [17]:
# Instantiate StandardScaler
ss_y = StandardScaler()

# Fit and transform train labels
y_train_scaled = ss_y.fit_transform(y_train)

# Transform validate and test labels
y_val_scaled = ss_y.transform(y_val)
y_test_scaled = ss_y.transform(y_test)

In the cell below: 
- Train the `normalized_model` on normalized input (`X_train`) and output (`y_train_scaled`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` as `(X_val, y_val_scaled)` 

In [18]:
# Model with all normalized inputs and outputs
np.random.seed(123)
normalized_model = Sequential()
normalized_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
normalized_model.add(layers.Dense(50, activation='relu'))
normalized_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_model.compile(optimizer='SGD', 
                         loss='mse', 
                         metrics=['mse']) 

# Train the model
normalized_model.fit(X_train, 
                     y_train_scaled, 
                     batch_size=32, 
                     epochs=150, 
                     validation_data=(X_val, y_val_scaled))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 1s 824us/step - loss: 0.4368 - mean_squared_error: 0.4368 - val_loss: 0.1991 - val_mean_squared_error: 0.1991
Epoch 2/150
1051/1051 [==============================] - 0s 221us/step - loss: 0.2273 - mean_squared_error: 0.2273 - val_loss: 0.1611 - val_mean_squared_error: 0.1611
Epoch 3/150
1051/1051 [==============================] - 0s 288us/step - loss: 0.1842 - mean_squared_error: 0.1842 - val_loss: 0.1589 - val_mean_squared_error: 0.1589
Epoch 4/150
1051/1051 [==============================] - 0s 283us/step - loss: 0.1643 - mean_squared_error: 0.1643 - val_loss: 0.1377 - val_mean_squared_error: 0.1377
Epoch 5/150
1051/1051 [==============================] - 0s 350us/step - loss: 0.1482 - mean_squared_error: 0.1482 - val_loss: 0.1320 - val_mean_squared_error: 0.1320
Epoch 6/150
1051/1051 [==============================] - 0s 313us/step - loss: 0.1329 - mean_squared_error: 0.1329 - v

1051/1051 [==============================] - 0s 261us/step - loss: 0.0328 - mean_squared_error: 0.0328 - val_loss: 0.1033 - val_mean_squared_error: 0.1033
Epoch 50/150
1051/1051 [==============================] - 0s 310us/step - loss: 0.0319 - mean_squared_error: 0.0319 - val_loss: 0.1015 - val_mean_squared_error: 0.1015
Epoch 51/150
1051/1051 [==============================] - 0s 225us/step - loss: 0.0318 - mean_squared_error: 0.0318 - val_loss: 0.1028 - val_mean_squared_error: 0.1028
Epoch 52/150
1051/1051 [==============================] - 0s 219us/step - loss: 0.0310 - mean_squared_error: 0.0310 - val_loss: 0.1031 - val_mean_squared_error: 0.1031
Epoch 53/150
1051/1051 [==============================] - 0s 159us/step - loss: 0.0303 - mean_squared_error: 0.0303 - val_loss: 0.1030 - val_mean_squared_error: 0.1030
Epoch 54/150
1051/1051 [==============================] - 0s 208us/step - loss: 0.0293 - mean_squared_error: 0.0293 - val_loss: 0.1045 - val_mean_squared_error: 0.1045
Epoch

Epoch 98/150
1051/1051 [==============================] - 0s 223us/step - loss: 0.0145 - mean_squared_error: 0.0145 - val_loss: 0.1099 - val_mean_squared_error: 0.1099
Epoch 99/150
1051/1051 [==============================] - 0s 196us/step - loss: 0.0144 - mean_squared_error: 0.0144 - val_loss: 0.1114 - val_mean_squared_error: 0.1114
Epoch 100/150
1051/1051 [==============================] - 0s 205us/step - loss: 0.0142 - mean_squared_error: 0.0142 - val_loss: 0.1106 - val_mean_squared_error: 0.1106
Epoch 101/150
1051/1051 [==============================] - 0s 216us/step - loss: 0.0141 - mean_squared_error: 0.0141 - val_loss: 0.1118 - val_mean_squared_error: 0.1118
Epoch 102/150
1051/1051 [==============================] - 0s 236us/step - loss: 0.0137 - mean_squared_error: 0.0137 - val_loss: 0.1111 - val_mean_squared_error: 0.1111
Epoch 103/150
1051/1051 [==============================] - 0s 204us/step - loss: 0.0137 - mean_squared_error: 0.0137 - val_loss: 0.1115 - val_mean_squared_er

1051/1051 [==============================] - 0s 344us/step - loss: 0.0085 - mean_squared_error: 0.0085 - val_loss: 0.1150 - val_mean_squared_error: 0.1150
Epoch 147/150
1051/1051 [==============================] - 0s 298us/step - loss: 0.0084 - mean_squared_error: 0.0084 - val_loss: 0.1153 - val_mean_squared_error: 0.1153
Epoch 148/150
1051/1051 [==============================] - 0s 314us/step - loss: 0.0082 - mean_squared_error: 0.0082 - val_loss: 0.1153 - val_mean_squared_error: 0.1153
Epoch 149/150
1051/1051 [==============================] - 0s 156us/step - loss: 0.0082 - mean_squared_error: 0.0082 - val_loss: 0.1151 - val_mean_squared_error: 0.1151
Epoch 150/150
1051/1051 [==============================] - 0s 107us/step - loss: 0.0082 - mean_squared_error: 0.0082 - val_loss: 0.1163 - val_mean_squared_error: 0.1163


Nicely done! After normalizing both the input and output, the model finally converged. 

- Evaluate the model (`normalized_model`) on training data (`X_train` and `y_train_scaled`) 

In [19]:
# Evaluate the model on training data
normalized_model.evaluate(X_train, y_train_scaled)

1051/1051 [==============================] - 0s 70us/step


[0.007847911796996017, 0.007847911796996017]

- Evaluate the model (`normalized_model`) on validate data (`X_val` and `y_val_scaled`) 

In [20]:
# Evaluate the model on validate data
normalized_model.evaluate(X_val, y_val_scaled)

263/263 [==============================] - 0s 103us/step


[0.11625553333260714, 0.11625553333260714]

Since the output is normalized, the metric above is not interpretable. To remedy this: 

- Generate predictions on validate data (`X_val`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_val` and `y_val_pred` 

In [21]:
# Generate predictions on validate data
y_val_pred_scaled = normalized_model.predict(X_val)

# Transform the predictions back to original scale
y_val_pred = ss_y.inverse_transform(y_val_pred_scaled)

# RMSE of validate data
np.sqrt(mean_squared_error(y_val, y_val_pred))

26794.04742089457

Great! Now that you have a converged model, you can also experiment with alternative optimizers and initialization strategies to see if you can find a better global minimum. (After all, the current models may have converged to a local minimum.) 

## Using Weight Initializers

In this section you will to use alternative initialization and optimization strategies. At the end, you'll then be asked to select the model which you believe performs the best.  

##  He Initialization

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='he_normal'`  

In [22]:
np.random.seed(123)
he_model = Sequential()

# Add the first hidden layer
he_model.add(layers.Dense(100, kernel_initializer='he_normal', activation='relu', input_shape=(n_features,)))

# Add another hidden layer
he_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
he_model.add(layers.Dense(1, activation='linear'))

# Compile the model
he_model.compile(optimizer='SGD', 
                 loss='mse', 
                 metrics=['mse'])

# Train the model
he_model.fit(X_train, 
             y_train_scaled, 
             batch_size=32, 
             epochs=150, 
             validation_data=(X_val, y_val_scaled))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 351us/step - loss: 0.5081 - mean_squared_error: 0.5081 - val_loss: 0.1698 - val_mean_squared_error: 0.1698
Epoch 2/150
1051/1051 [==============================] - 0s 163us/step - loss: 0.2217 - mean_squared_error: 0.2217 - val_loss: 0.1430 - val_mean_squared_error: 0.1430
Epoch 3/150
1051/1051 [==============================] - 0s 136us/step - loss: 0.1797 - mean_squared_error: 0.1797 - val_loss: 0.1379 - val_mean_squared_error: 0.1379
Epoch 4/150
1051/1051 [==============================] - 0s 147us/step - loss: 0.1568 - mean_squared_error: 0.1568 - val_loss: 0.1248 - val_mean_squared_error: 0.1248
Epoch 5/150
1051/1051 [==============================] - 0s 146us/step - loss: 0.1410 - mean_squared_error: 0.1410 - val_loss: 0.1193 - val_mean_squared_error: 0.1193
Epoch 6/150
1051/1051 [==============================] - 0s 148us/step - loss: 0.1266 - mean_squared_error: 0.1266 - v

Epoch 50/150
1051/1051 [==============================] - 0s 105us/step - loss: 0.0304 - mean_squared_error: 0.0304 - val_loss: 0.1084 - val_mean_squared_error: 0.1084
Epoch 51/150
1051/1051 [==============================] - 0s 82us/step - loss: 0.0297 - mean_squared_error: 0.0297 - val_loss: 0.1087 - val_mean_squared_error: 0.1087
Epoch 52/150
1051/1051 [==============================] - 0s 88us/step - loss: 0.0293 - mean_squared_error: 0.0293 - val_loss: 0.1098 - val_mean_squared_error: 0.1098
Epoch 53/150
1051/1051 [==============================] - 0s 84us/step - loss: 0.0288 - mean_squared_error: 0.0288 - val_loss: 0.1099 - val_mean_squared_error: 0.1099
Epoch 54/150
1051/1051 [==============================] - 0s 90us/step - loss: 0.0282 - mean_squared_error: 0.0282 - val_loss: 0.1102 - val_mean_squared_error: 0.1102
Epoch 55/150
1051/1051 [==============================] - 0s 87us/step - loss: 0.0279 - mean_squared_error: 0.0279 - val_loss: 0.1097 - val_mean_squared_error: 0.10

1051/1051 [==============================] - 0s 92us/step - loss: 0.0145 - mean_squared_error: 0.0145 - val_loss: 0.1189 - val_mean_squared_error: 0.1189
Epoch 100/150
1051/1051 [==============================] - 0s 93us/step - loss: 0.0142 - mean_squared_error: 0.0142 - val_loss: 0.1183 - val_mean_squared_error: 0.1183
Epoch 101/150
1051/1051 [==============================] - 0s 86us/step - loss: 0.0140 - mean_squared_error: 0.0140 - val_loss: 0.1183 - val_mean_squared_error: 0.1183
Epoch 102/150
1051/1051 [==============================] - 0s 88us/step - loss: 0.0140 - mean_squared_error: 0.0140 - val_loss: 0.1175 - val_mean_squared_error: 0.1175
Epoch 103/150
1051/1051 [==============================] - 0s 88us/step - loss: 0.0139 - mean_squared_error: 0.0139 - val_loss: 0.1177 - val_mean_squared_error: 0.1177
Epoch 104/150
1051/1051 [==============================] - 0s 75us/step - loss: 0.0136 - mean_squared_error: 0.0136 - val_loss: 0.1201 - val_mean_squared_error: 0.1201
Epoch 

Epoch 148/150
1051/1051 [==============================] - 0s 87us/step - loss: 0.0086 - mean_squared_error: 0.0086 - val_loss: 0.1237 - val_mean_squared_error: 0.1237
Epoch 149/150
1051/1051 [==============================] - 0s 76us/step - loss: 0.0085 - mean_squared_error: 0.0085 - val_loss: 0.1232 - val_mean_squared_error: 0.1232
Epoch 150/150
1051/1051 [==============================] - 0s 87us/step - loss: 0.0084 - mean_squared_error: 0.0084 - val_loss: 0.1238 - val_mean_squared_error: 0.1238


Evaluate the model (`he_model`) on training data (`X_train` and `y_train_scaled`) 

In [23]:
# Evaluate the model on training data
he_model.evaluate(X_train, y_train_scaled)

1051/1051 [==============================] - 0s 58us/step


[0.008269403494015464, 0.008269403494015464]

Evaluate the model (`he_model`) on validate data (`X_train` and `y_train_scaled`) 

In [24]:
# Evaluate the model on validate data
he_model.evaluate(X_val, y_val_scaled)

263/263 [==============================] - 0s 86us/step


[0.12382162206526491, 0.12382162206526491]

## Lecun Initialization 

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='lecun_normal'` 

In [25]:
np.random.seed(123)
lecun_model = Sequential()

# Add the first hidden layer
lecun_model.add(layers.Dense(100, kernel_initializer='lecun_normal', activation='relu', input_shape=(n_features,)))

# Add another hidden layer
lecun_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
lecun_model.add(layers.Dense(1, activation='linear'))

# Compile the model
lecun_model.compile(optimizer='SGD', 
                    loss='mse', 
                    metrics=['mse'])

# Train the model
lecun_model.fit(X_train, 
                y_train_scaled, 
                batch_size=32, 
                epochs=150, 
                validation_data=(X_val, y_val_scaled))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 310us/step - loss: 0.4771 - mean_squared_error: 0.4771 - val_loss: 0.1462 - val_mean_squared_error: 0.1462
Epoch 2/150
1051/1051 [==============================] - 0s 133us/step - loss: 0.2096 - mean_squared_error: 0.2096 - val_loss: 0.1231 - val_mean_squared_error: 0.1231
Epoch 3/150
1051/1051 [==============================] - 0s 138us/step - loss: 0.1698 - mean_squared_error: 0.1698 - val_loss: 0.1168 - val_mean_squared_error: 0.1168
Epoch 4/150
1051/1051 [==============================] - 0s 144us/step - loss: 0.1478 - mean_squared_error: 0.1478 - val_loss: 0.1085 - val_mean_squared_error: 0.1085
Epoch 5/150
1051/1051 [==============================] - 0s 152us/step - loss: 0.1330 - mean_squared_error: 0.1330 - val_loss: 0.1059 - val_mean_squared_error: 0.1059
Epoch 6/150
1051/1051 [==============================] - 0s 174us/step - loss: 0.1197 - mean_squared_error: 0.1197 - v

1051/1051 [==============================] - 0s 94us/step - loss: 0.0278 - mean_squared_error: 0.0278 - val_loss: 0.1190 - val_mean_squared_error: 0.1190
Epoch 50/150
1051/1051 [==============================] - 0s 166us/step - loss: 0.0268 - mean_squared_error: 0.0268 - val_loss: 0.1158 - val_mean_squared_error: 0.1158
Epoch 51/150
1051/1051 [==============================] - 0s 84us/step - loss: 0.0263 - mean_squared_error: 0.0263 - val_loss: 0.1169 - val_mean_squared_error: 0.1169
Epoch 52/150
1051/1051 [==============================] - 0s 83us/step - loss: 0.0260 - mean_squared_error: 0.0260 - val_loss: 0.1184 - val_mean_squared_error: 0.1184
Epoch 53/150
1051/1051 [==============================] - 0s 92us/step - loss: 0.0254 - mean_squared_error: 0.0254 - val_loss: 0.1190 - val_mean_squared_error: 0.1190
Epoch 54/150
1051/1051 [==============================] - 0s 95us/step - loss: 0.0248 - mean_squared_error: 0.0248 - val_loss: 0.1186 - val_mean_squared_error: 0.1186
Epoch 55/1

1051/1051 [==============================] - 0s 96us/step - loss: 0.0129 - mean_squared_error: 0.0129 - val_loss: 0.1269 - val_mean_squared_error: 0.1269
Epoch 99/150
1051/1051 [==============================] - 0s 128us/step - loss: 0.0127 - mean_squared_error: 0.0127 - val_loss: 0.1296 - val_mean_squared_error: 0.1296
Epoch 100/150
1051/1051 [==============================] - 0s 158us/step - loss: 0.0125 - mean_squared_error: 0.0125 - val_loss: 0.1287 - val_mean_squared_error: 0.1287
Epoch 101/150
1051/1051 [==============================] - 0s 128us/step - loss: 0.0125 - mean_squared_error: 0.0125 - val_loss: 0.1293 - val_mean_squared_error: 0.1293
Epoch 102/150
1051/1051 [==============================] - 0s 93us/step - loss: 0.0124 - mean_squared_error: 0.0124 - val_loss: 0.1275 - val_mean_squared_error: 0.1275
Epoch 103/150
1051/1051 [==============================] - 0s 122us/step - loss: 0.0121 - mean_squared_error: 0.0121 - val_loss: 0.1283 - val_mean_squared_error: 0.1283
Epo

Epoch 147/150
1051/1051 [==============================] - 0s 109us/step - loss: 0.0074 - mean_squared_error: 0.0074 - val_loss: 0.1346 - val_mean_squared_error: 0.1346
Epoch 148/150
1051/1051 [==============================] - 0s 92us/step - loss: 0.0073 - mean_squared_error: 0.0073 - val_loss: 0.1357 - val_mean_squared_error: 0.1357
Epoch 149/150
1051/1051 [==============================] - 0s 94us/step - loss: 0.0073 - mean_squared_error: 0.0073 - val_loss: 0.1339 - val_mean_squared_error: 0.1339
Epoch 150/150
1051/1051 [==============================] - 0s 88us/step - loss: 0.0073 - mean_squared_error: 0.0073 - val_loss: 0.1353 - val_mean_squared_error: 0.1353


Evaluate the model (`lecun_model`) on training data (`X_train` and `y_train_scaled`) 

In [26]:
# Evaluate the model on training data
lecun_model.evaluate(X_train, y_train_scaled)

1051/1051 [==============================] - 0s 92us/step


[0.007084393265039494, 0.007084393265039494]

Evaluate the model (`lecun_model`) on validate data (`X_train` and `y_train_scaled`) 

In [27]:
# Evaluate the model on validate data
lecun_model.evaluate(X_val, y_val_scaled)

263/263 [==============================] - 0s 85us/step


[0.13529217650426884, 0.13529217650426884]

Not much of a difference, but a useful note to consider when tuning your network. Next, let's investigate the impact of various optimization algorithms.

## RMSprop 

Compile the `rmsprop_model` with: 

- `'rmsprop'` as the optimizer 
- track `'mse'` as the loss and metric  

In [32]:
np.random.seed(123)
rmsprop_model = Sequential()
rmsprop_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
rmsprop_model.add(layers.Dense(50, activation='relu'))
rmsprop_model.add(layers.Dense(1, activation='linear'))

# Compile the model
rmsprop_model.compile(optimizer='rmsprop', loss='mse', metrics=['mse'])

# Train the model
rmsprop_model.fit(X_train, 
                  y_train_scaled, 
                  batch_size=32, 
                  epochs=150, 
                  validation_data=(X_val, y_val_scaled))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 1s 678us/step - loss: 0.3207 - mean_squared_error: 0.3207 - val_loss: 0.1386 - val_mean_squared_error: 0.1386
Epoch 2/150
1051/1051 [==============================] - 0s 128us/step - loss: 0.1669 - mean_squared_error: 0.1669 - val_loss: 0.1112 - val_mean_squared_error: 0.1112
Epoch 3/150
1051/1051 [==============================] - 0s 108us/step - loss: 0.1243 - mean_squared_error: 0.1243 - val_loss: 0.1298 - val_mean_squared_error: 0.1298
Epoch 4/150
1051/1051 [==============================] - 0s 106us/step - loss: 0.0983 - mean_squared_error: 0.0983 - val_loss: 0.1078 - val_mean_squared_error: 0.1078
Epoch 5/150
1051/1051 [==============================] - 0s 132us/step - loss: 0.0891 - mean_squared_error: 0.0891 - val_loss: 0.1020 - val_mean_squared_error: 0.1020
Epoch 6/150
1051/1051 [==============================] - 0s 252us/step - loss: 0.0684 - mean_squared_error: 0.0684 - v

1051/1051 [==============================] - 0s 89us/step - loss: 0.0086 - mean_squared_error: 0.0086 - val_loss: 0.0993 - val_mean_squared_error: 0.0993
Epoch 50/150
1051/1051 [==============================] - 0s 87us/step - loss: 0.0083 - mean_squared_error: 0.0083 - val_loss: 0.1200 - val_mean_squared_error: 0.1200
Epoch 51/150
1051/1051 [==============================] - 0s 97us/step - loss: 0.0071 - mean_squared_error: 0.0071 - val_loss: 0.1103 - val_mean_squared_error: 0.1103
Epoch 52/150
1051/1051 [==============================] - 0s 101us/step - loss: 0.0102 - mean_squared_error: 0.0102 - val_loss: 0.1023 - val_mean_squared_error: 0.1023
Epoch 53/150
1051/1051 [==============================] - 0s 92us/step - loss: 0.0081 - mean_squared_error: 0.0081 - val_loss: 0.0883 - val_mean_squared_error: 0.0883
Epoch 54/150
1051/1051 [==============================] - 0s 92us/step - loss: 0.0068 - mean_squared_error: 0.0068 - val_loss: 0.0946 - val_mean_squared_error: 0.0946
Epoch 55/1

1051/1051 [==============================] - 0s 83us/step - loss: 0.0051 - mean_squared_error: 0.0051 - val_loss: 0.0964 - val_mean_squared_error: 0.0964
Epoch 99/150
1051/1051 [==============================] - 0s 91us/step - loss: 0.0055 - mean_squared_error: 0.0055 - val_loss: 0.0913 - val_mean_squared_error: 0.0913
Epoch 100/150
1051/1051 [==============================] - 0s 94us/step - loss: 0.0051 - mean_squared_error: 0.0051 - val_loss: 0.0938 - val_mean_squared_error: 0.0938
Epoch 101/150
1051/1051 [==============================] - 0s 82us/step - loss: 0.0050 - mean_squared_error: 0.0050 - val_loss: 0.1032 - val_mean_squared_error: 0.1032
Epoch 102/150
1051/1051 [==============================] - 0s 93us/step - loss: 0.0047 - mean_squared_error: 0.0047 - val_loss: 0.0946 - val_mean_squared_error: 0.0946
Epoch 103/150
1051/1051 [==============================] - 0s 94us/step - loss: 0.0061 - mean_squared_error: 0.0061 - val_loss: 0.1030 - val_mean_squared_error: 0.1030
Epoch 1

Epoch 147/150
1051/1051 [==============================] - 0s 93us/step - loss: 0.0043 - mean_squared_error: 0.0043 - val_loss: 0.0958 - val_mean_squared_error: 0.0958
Epoch 148/150
1051/1051 [==============================] - 0s 85us/step - loss: 0.0036 - mean_squared_error: 0.0036 - val_loss: 0.1045 - val_mean_squared_error: 0.1045
Epoch 149/150
1051/1051 [==============================] - 0s 104us/step - loss: 0.0036 - mean_squared_error: 0.0036 - val_loss: 0.0932 - val_mean_squared_error: 0.0932
Epoch 150/150
1051/1051 [==============================] - 0s 89us/step - loss: 0.0041 - mean_squared_error: 0.0041 - val_loss: 0.1018 - val_mean_squared_error: 0.1018


Evaluate the model (`rmsprop_model`) on training data (`X_train` and `y_train_scaled`) 

In [33]:
# Evaluate the model on training data
rmsprop_model.evaluate(X_train, y_train_scaled)

1051/1051 [==============================] - 0s 81us/step


[0.005507833463938235, 0.005507833463938235]

Evaluate the model (`rmsprop_model`) on training data (`X_train` and `y_train_scaled`) 

In [34]:
# Evaluate the model on validate data
rmsprop_model.evaluate(X_val, y_val_scaled)

263/263 [==============================] - 0s 175us/step


[0.10177629234219686, 0.10177629234219686]

## Adam 

Compile the `adam_model` with: 

- `'Adam'` as the optimizer 
- track `'mse'` as the loss and metric  

In [35]:
np.random.seed(123)
adam_model = Sequential()
adam_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
adam_model.add(layers.Dense(50, activation='relu'))
adam_model.add(layers.Dense(1, activation='linear'))

# Compile the model
adam_model.compile(optimizer='Adam',
                   loss='mse',
                   metrics=['mse'])

# Train the model
adam_model.fit(X_train, 
               y_train_scaled, 
               batch_size=32, 
               epochs=150, 
               validation_data=(X_val, y_val_scaled))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 459us/step - loss: 0.3803 - mean_squared_error: 0.3803 - val_loss: 0.1651 - val_mean_squared_error: 0.1651
Epoch 2/150
1051/1051 [==============================] - 0s 123us/step - loss: 0.1611 - mean_squared_error: 0.1611 - val_loss: 0.1334 - val_mean_squared_error: 0.1334
Epoch 3/150
1051/1051 [==============================] - 0s 142us/step - loss: 0.1129 - mean_squared_error: 0.1129 - val_loss: 0.1047 - val_mean_squared_error: 0.1047
Epoch 4/150
1051/1051 [==============================] - 0s 119us/step - loss: 0.0860 - mean_squared_error: 0.0860 - val_loss: 0.1012 - val_mean_squared_error: 0.1012
Epoch 5/150
1051/1051 [==============================] - 0s 139us/step - loss: 0.0740 - mean_squared_error: 0.0740 - val_loss: 0.1015 - val_mean_squared_error: 0.1015
Epoch 6/150
1051/1051 [==============================] - 0s 128us/step - loss: 0.0569 - mean_squared_error: 0.0569 - v

1051/1051 [==============================] - 0s 202us/step - loss: 0.0033 - mean_squared_error: 0.0033 - val_loss: 0.0985 - val_mean_squared_error: 0.0985
Epoch 50/150
1051/1051 [==============================] - 0s 154us/step - loss: 0.0030 - mean_squared_error: 0.0030 - val_loss: 0.1058 - val_mean_squared_error: 0.1058
Epoch 51/150
1051/1051 [==============================] - 0s 108us/step - loss: 0.0023 - mean_squared_error: 0.0023 - val_loss: 0.0960 - val_mean_squared_error: 0.0960
Epoch 52/150
1051/1051 [==============================] - 0s 104us/step - loss: 0.0021 - mean_squared_error: 0.0021 - val_loss: 0.1034 - val_mean_squared_error: 0.1034
Epoch 53/150
1051/1051 [==============================] - 0s 95us/step - loss: 0.0021 - mean_squared_error: 0.0021 - val_loss: 0.0973 - val_mean_squared_error: 0.0973
Epoch 54/150
1051/1051 [==============================] - 0s 88us/step - loss: 0.0017 - mean_squared_error: 0.0017 - val_loss: 0.0970 - val_mean_squared_error: 0.0970
Epoch 5

1051/1051 [==============================] - 0s 99us/step - loss: 0.0012 - mean_squared_error: 0.0012 - val_loss: 0.0928 - val_mean_squared_error: 0.0928
Epoch 99/150
1051/1051 [==============================] - 0s 99us/step - loss: 0.0012 - mean_squared_error: 0.0012 - val_loss: 0.0917 - val_mean_squared_error: 0.0917
Epoch 100/150
1051/1051 [==============================] - 0s 95us/step - loss: 8.7228e-04 - mean_squared_error: 8.7228e-04 - val_loss: 0.0944 - val_mean_squared_error: 0.0944
Epoch 101/150
1051/1051 [==============================] - 0s 94us/step - loss: 7.9866e-04 - mean_squared_error: 7.9866e-04 - val_loss: 0.0898 - val_mean_squared_error: 0.0898
Epoch 102/150
1051/1051 [==============================] - 0s 93us/step - loss: 0.0012 - mean_squared_error: 0.0012 - val_loss: 0.0938 - val_mean_squared_error: 0.0938
Epoch 103/150
1051/1051 [==============================] - 0s 91us/step - loss: 0.0021 - mean_squared_error: 0.0021 - val_loss: 0.0899 - val_mean_squared_error

Epoch 146/150
1051/1051 [==============================] - 0s 98us/step - loss: 0.0073 - mean_squared_error: 0.0073 - val_loss: 0.0940 - val_mean_squared_error: 0.0940
Epoch 147/150
1051/1051 [==============================] - 0s 95us/step - loss: 0.0079 - mean_squared_error: 0.0079 - val_loss: 0.0917 - val_mean_squared_error: 0.0917
Epoch 148/150
1051/1051 [==============================] - 0s 89us/step - loss: 0.0104 - mean_squared_error: 0.0104 - val_loss: 0.0956 - val_mean_squared_error: 0.0956
Epoch 149/150
1051/1051 [==============================] - 0s 95us/step - loss: 0.0087 - mean_squared_error: 0.0087 - val_loss: 0.0962 - val_mean_squared_error: 0.0962
Epoch 150/150
1051/1051 [==============================] - 0s 111us/step - loss: 0.0072 - mean_squared_error: 0.0072 - val_loss: 0.0922 - val_mean_squared_error: 0.0922


Evaluate the model (`adam_model`) on training data (`X_train` and `y_train_scaled`) 

In [40]:
# Evaluate the model on training data
adam_model.evaluate(X_train, y_train_scaled)

1051/1051 [==============================] - 0s 115us/step


[0.006434062645140571, 0.006434062645140571]

Evaluate the model (`adam_model`) on training data (`X_train` and `y_train_scaled`) 

In [41]:
# Evaluate the model on validate data
adam_model.evaluate(X_val, y_val_scaled)

263/263 [==============================] - 0s 107us/step


[0.09220970667785339, 0.09220970667785339]

## Select a Final Model

Now, select the model with the best performance based on the training and validation sets. Evaluate this top model using the test set!

In [42]:
# Evaluate the best model on test data
rmsprop_model.evaluate(X_test, y_test_scaled)

146/146 [==============================] - 0s 378us/step


[0.2169951216815269, 0.2169951216815269]

As earlier, this metric is hard to interpret because the output is scaled. 

- Generate predictions on test data (`X_test`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_test` and `y_test_pred` 

In [43]:
# Generate predictions on test data
y_test_pred_scaled = rmsprop_model.predict(X_test)

# Transform the predictions back to original scale
y_test_pred = ss_y.inverse_transform(y_test_pred_scaled)

# MSE of test data
np.sqrt(mean_squared_error(y_test, y_test_pred))

36606.35480238535

## Summary  

In this lab, you worked to ensure your model converged properly by normalizing both the input and output. Additionally, you also investigated the impact of varying initialization and optimization routines.